In [5]:
import pandas as pd

# Carrega o csv

In [11]:
file_path = 'files/pre-processed.csv'
df = pd.read_csv(file_path)

# Cria o arquivo .tsv

In [12]:
corpus_df = df[['preprocessed_news', 'index', 'label']]
corpus_file_path = 'files/corpus.tsv'
corpus_df.to_csv(corpus_file_path, sep='\t', index=False, header=False)

# Extrai palavras únicas criando o vocabulário

In [13]:
vocabulary = set()
df['preprocessed_news'].str.split().apply(vocabulary.update)
vocabulary_file_path = 'files/vocabulary.txt'
with open(vocabulary_file_path, 'w') as vocab_file:
    vocab_file.write('\n'.join(sorted(vocabulary)))